In [1]:
using Pkg
Pkg.activate()
Pkg.instantiate()
#Carico le librerie utili all'implementazione ed alla fase di test
using Profile
using ProfileView
using BenchmarkTools #utile per il test
using SparseArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
using NearestNeighbors
Lar = LinearAlgebraicRepresentation
using ViewerGL
GL = ViewerGL

  Activating project at `~/.julia/environments/v1.7`


ViewerGL

In [2]:
store = [];
scaling = 1.5;
V,(VV,EV,FV,CV) = Lar.cuboid([0.25,0.25,0.25],true,[-0.25,-0.25,-0.25]);
mybox = (V,CV,FV,EV);
for k=1:5
	size = rand()*scaling
	scale = Lar.s(size,size,size)
	transl = Lar.t(rand(3)...)
	alpha = 2*pi*rand()
	rx = Lar.r(alpha,0,0); ry = Lar.r(0,alpha,0); rz = Lar.r(0,0,alpha)
	rot = rx * ry * rz
	str = Lar.Struct([ transl, scale, rot, mybox ])
	obj = Lar.struct2lar(str)
	vs = obj[1]
	diag = LinearAlgebra.norm(vs[:,8]-vs[:,1])
	if diag > 1/5
		push!(store, obj)
	end
end

str = Lar.Struct(store);
V,CV,FV,EV = Lar.struct2lar(str);
cop_EV = Lar.coboundary_0(EV::Lar.Cells);
cop_FE = Lar.coboundary_1(V, FV::Lar.Cells, EV::Lar.Cells);
W = convert(Lar.Points, V');

In [3]:
function spatial_arrangement_1_rev(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

	# initializations
	fs_num = size(copFE, 1)
    rV = Array{Float64,2}(undef,0,3)
    rEV = SparseArrays.spzeros(Int8,0,0)
    rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        @async for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = Lar.Arrangement.frag_face(V, copEV, copFE, sp_idx, sigma)
# 			v = size(nV,1); e = nEV.m; f = nFE.m
# @show v-e+f
# 			if v-e+f > 1
# 				g = v-e+f - 1	# number of inner loops
# 				nFE = removeinnerloops(g, nFE)
# @show nV;
# @show SparseArrays.nzind(nEV)
# @show SparseArrays.nzind(nFE)
# 			end
			#nV, nEV, nFE = Lar.fragface(V, copEV, copFE, sp_idx, sigma)
			nV = convert(Lar.Points, nV)
            a,b,c = Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            rV=a;  rEV=b;  rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = Lar.Arrangement.merge_vertices(rV, rEV, rFE)
    return rV, rEV, rFE
end

spatial_arrangement_1_rev (generic function with 2 methods)

In [ ]:
@btime Lar.Arrangement.spatial_arrangement_1(W,cop_EV,cop_FE,true )

In [5]:
@btime spatial_arrangement_1_rev(W,cop_EV,cop_FE,false)

  617.084 μs (13360 allocations: 517.37 KiB)


(Matrix{Float64}(undef, 0, 3), sparse(Int64[], Int64[], Int8[], 0, 0), sparse(Int64[], Int64[], Int8[], 0, 0))